### All days of the challange:

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [188]:
# libraries we'll use

import pandas as pd
# read in all our data NFL nfl1-> Play by Play 2009-2017 (v4), nfl2 -> NFL Play by Play 2009-2016 (v3)
nfl1 = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv",index_col = 'GameID')
nfl2 = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv",index_col = 'GameID')



C:\Users\marwa\AppData\Local\Temp\ipykernel_4480\3271162948.py:5: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl1 = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv",index_col = 'GameID')
C:\Users\marwa\AppData\Local\Temp\ipykernel_4480\3271162948.py:6: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl2 = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv",index_col = 'GameID')


The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [189]:
# look at a few rows of the nfl1_data file. I can see a handful of missing data already!
nfl1.head()

,Date,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
GameID,,,,,,,,,,,,,,,,,,,,,
2009091000,2009-09-10,1,1,NaN,15:00,15,3600.0,0.0,TEN,30.0,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
2009091000,2009-09-10,1,1,1.0,14:53,15,3593.0,7.0,PIT,42.0,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2009091000,2009-09-10,1,1,2.0,14:16,15,3556.0,37.0,PIT,47.0,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
2009091000,2009-09-10,1,1,3.0,13:35,14,3515.0,41.0,PIT,44.0,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
2009091000,2009-09-10,1,1,4.0,13:27,14,3507.0,8.0,PIT,44.0,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Yep, it looks like there's some missing values. What about in the nfl2 dataset?

In [171]:
# your turn! Look at a couple of rows from the nfl2 dataset. Do you notice any missing data?
# your code goes here :)
nfl2.head(2)

,Date,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
GameID,,,,,,,,,,,,,,,,,,,,,
2009091000,2009-09-10,1,1,NaN,15:00,15,3600.0,0.0,TEN,30.0,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
2009091000,2009-09-10,1,1,1.0,14:53,15,3593.0,7.0,PIT,42.0,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009


# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 

In [194]:
# get the number of missing data points per column
nfl1.isna().sum()

# look at the # of missing points in the first ten columns
nfl1[nfl1.columns[:10]].isna().sum()

Date                0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
yrdln             840
dtype: int64

That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [173]:
# how many total missing values do we have?
nfl1.isna().sum().sum()

# percent of data that is missing
nfl1.isna().sum().sum() / (len(nfl1) * len(nfl1.columns)) * 100


27.941156613453778

Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [174]:
# your turn! Find out what percent of the nfl2_ dataset is missing
nfl2.isna().sum().sum()
nfl2.isna().sum().sum() / (len(nfl2) * len(nfl2.columns)) * 100

27.926052254222377

# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

In [175]:
# look at the # of missing points in the first ten columns
nfl1.iloc[:,:10].isna().sum()

Date                0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
yrdln             840
dtype: int64

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [176]:
# remove all the rows that contain a missing value
nfl1.dropna()

,Date,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
GameID,,,,,,,,,,,,,,,,,,,,,


Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [177]:
# remove all columns with at least one missing value
nfl1.dropna(axis =1)

,Date,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,Touchdown,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
GameID,,,,,,,,,,,,,,,,,,,,,
2009091000,2009-09-10,1,1,15,0,0,1,39,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2009091000,2009-09-10,1,1,15,10,5,1,5,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2009091000,2009-09-10,1,1,15,5,2,1,-3,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2009091000,2009-09-10,1,1,14,8,2,1,0,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2009091000,2009-09-10,1,1,14,8,2,1,0,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017123101,2017-12-31,29,4,1,0,-4,1,0,0,0,...,CIN,1,0,3,0,2,0,0.0,0.0,2017
2017123101,2017-12-31,29,4,1,14,-4,1,0,0,0,...,CIN,0,2,2,0,2,0,0.0,0.0,2017
2017123101,2017-12-31,29,4,1,14,9,1,13,0,0,...,CIN,0,2,2,0,2,0,0.0,0.0,2017


In [178]:
# just how much data did we lose?
len(nfl1) * len(nfl1.columns)  - len(nfl1.dropna(axis =1)) * len(nfl1.dropna(axis =1).columns) 

26499720

We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

In [179]:
# Your turn! Try removing all the rows from the nfl2 dataset that contain missing values. How many are left?
nfl2.dropna()

,Date,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
GameID,,,,,,,,,,,,,,,,,,,,,


In [180]:
# Now try removing all the columns with empty values. Now how much of your data is left?
nfl2.dropna(axis=1)

,Date,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,Touchdown,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
GameID,,,,,,,,,,,,,,,,,,,,,
2009091000,2009-09-10,1,1,15,0,0,1,39,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
2009091000,2009-09-10,1,1,15,10,5,1,5,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
2009091000,2009-09-10,1,1,15,5,2,1,-3,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
2009091000,2009-09-10,1,1,14,8,2,1,0,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
2009091000,2009-09-10,1,1,14,8,2,1,0,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017010102,2017-01-01,20,4,1,10,35,1,35,1,1,...,GB,0,0,0,0,0,0,0.000000,0.0,2016
2017010102,2017-01-01,20,4,1,0,35,1,0,1,0,...,GB,0,0,0,0,0,0,0.931115,0.0,2016
2017010102,2017-01-01,21,4,1,0,0,1,0,0,0,...,GB,0,0,0,0,0,0,0.000000,0.0,2016


# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [181]:
# get a small subset of the NFL dataset
nfl1_sample = nfl1.sample(frac = 0.0001)
nfl1_sample

,Date,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
GameID,,,,,,,,,,,,,,,,,,,,,
2009120700,2009-12-07,10,2,NaN,04:35,5,2075.0,0.0,GB,30.0,...,NaN,0.816921,0.183079,0.811510,0.188490,0.183079,0.005411,NaN,NaN,2009
2014110905,2014-11-09,18,4,1.0,03:32,4,212.0,29.0,PIT,36.0,...,0.000000,0.964700,0.035300,0.965685,0.034315,0.035300,-0.000985,-0.000985,0.000000,2014
2012010110,2012-01-01,1,1,2.0,10:09,11,3309.0,46.0,TB,16.0,...,NaN,0.621752,0.378248,0.621955,0.378045,0.621752,0.000203,NaN,NaN,2011
2016112004,2016-11-20,1,1,3.0,13:49,14,3529.0,31.0,TEN,29.0,...,-0.377078,0.504992,0.495008,0.564111,0.435889,0.495008,-0.059119,-0.047761,-0.011359,2016
2016122408,2016-12-24,9,2,2.0,03:34,4,2014.0,5.0,IND,41.0,...,NaN,0.664430,0.335570,0.640283,0.359717,0.335570,0.024146,NaN,NaN,2016
2009110800,2009-11-08,17,4,4.0,02:34,3,154.0,7.0,WAS,43.0,...,-6.058177,0.980189,0.019811,0.989043,0.010957,0.019811,-0.008853,0.026943,-0.035796,2009
2016112711,2016-11-27,16,3,2.0,00:42,1,942.0,4.0,NE,22.0,...,0.830430,0.634710,0.365290,0.645529,0.354471,0.365290,-0.010819,-0.040210,0.029392,2016
2010122602,2010-12-26,12,3,1.0,12:40,13,1660.0,39.0,CLE,44.0,...,NaN,0.239004,0.760996,0.235103,0.764897,0.239004,-0.003901,NaN,NaN,2010
2014122100,2014-12-21,17,4,NaN,09:59,10,599.0,12.0,CAR,2.0,...,NaN,0.409627,0.590373,0.403564,0.596436,0.590373,0.006064,NaN,NaN,2014


We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [182]:
# replace all NA's with 0
nfl1_sample.fillna(0)

,Date,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
GameID,,,,,,,,,,,,,,,,,,,,,
2009120700,2009-12-07,10,2,0.0,04:35,5,2075.0,0.0,GB,30.0,...,0.000000,0.816921,0.183079,0.811510,0.188490,0.183079,0.005411,0.000000,0.000000,2009
2014110905,2014-11-09,18,4,1.0,03:32,4,212.0,29.0,PIT,36.0,...,0.000000,0.964700,0.035300,0.965685,0.034315,0.035300,-0.000985,-0.000985,0.000000,2014
2012010110,2012-01-01,1,1,2.0,10:09,11,3309.0,46.0,TB,16.0,...,0.000000,0.621752,0.378248,0.621955,0.378045,0.621752,0.000203,0.000000,0.000000,2011
2016112004,2016-11-20,1,1,3.0,13:49,14,3529.0,31.0,TEN,29.0,...,-0.377078,0.504992,0.495008,0.564111,0.435889,0.495008,-0.059119,-0.047761,-0.011359,2016
2016122408,2016-12-24,9,2,2.0,03:34,4,2014.0,5.0,IND,41.0,...,0.000000,0.664430,0.335570,0.640283,0.359717,0.335570,0.024146,0.000000,0.000000,2016
2009110800,2009-11-08,17,4,4.0,02:34,3,154.0,7.0,WAS,43.0,...,-6.058177,0.980189,0.019811,0.989043,0.010957,0.019811,-0.008853,0.026943,-0.035796,2009
2016112711,2016-11-27,16,3,2.0,00:42,1,942.0,4.0,NE,22.0,...,0.830430,0.634710,0.365290,0.645529,0.354471,0.365290,-0.010819,-0.040210,0.029392,2016
2010122602,2010-12-26,12,3,1.0,12:40,13,1660.0,39.0,CLE,44.0,...,0.000000,0.239004,0.760996,0.235103,0.764897,0.239004,-0.003901,0.000000,0.000000,2010
2014122100,2014-12-21,17,4,0.0,09:59,10,599.0,12.0,CAR,2.0,...,0.000000,0.409627,0.590373,0.403564,0.596436,0.590373,0.006064,0.000000,0.000000,2014


I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [187]:
# replace all NA's the value that comes directly after it in the same column,
nfl1_sample = nfl1_sample.bfill()
nfl1_sample
# then replace all the reamining na's with 0
nfl1_sample = nfl1_sample.fillna(0)

Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). First, however, why don't you try replacing some of the missing values in the sf_permit dataset?